# Import Libs

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Load the OpenAI key

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


# Preparing a System Prompt

In [4]:
system_message = "You are a helpful translator that translate sentences/words from Moroccan Darija to (English / Arabic / French) or the opposite. "
system_message = "The translate should be by the meaning of the sentence and not by words, for example: 'زيد نقصط' or 'zid n9es' --> 'decrease more' not 'add less'. "
system_message = "The answer should only be the response sentence or word with no explaination at all. "
system_message = "A message example: \n"
system_message = """
    1- translated from Darija to English: [Bghit Nakul]
    2- translated from Darija to English: [بغيت الناكل]
    3- translated from English to Darija: [I wanna eat]\n
"""
system_message = "Your response must be a snetence for example: \n"
system_message = """
    1- I wanna eat
    2- I want to eat
    3- bghit nakul or بغيت الناكل
"""
system_message = "\nImportant NOTE!!! Don't not write explaination only and only the tranlation of the sentence to the target language."

# AI Chat Rresponse

In [5]:
def chat(message, source, target):
    userMessage = f'translated from {source} to {target}: [{message}]'
    
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": userMessage}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

# Start the chat interface

In [14]:
def create_translator_interface():
    """
    Create the Gradio interface with dropdowns for language selection
    """
    
    # Define language options
    language_options = ["Arabic", "English", "Darija", "French"]
    
    # Create the interface using gr.Blocks for more control
    with gr.Blocks(title="Darija Translator") as interface:
         gr.Markdown("# Darija Translator")
         gr.Markdown("Enter text and select source and target languages")
        
         with gr.Row():
             with gr.Column():
                # Input section
                gr.Markdown("### Input")
                input_text = gr.Textbox(
                    label="Enter text to translate",
                    placeholder="Type your text here...",
                    lines=3
                )
                source_lang = gr.Dropdown(
                    choices=language_options,
                    label="Source Language",
                    value="English"  # default value
                )
            
             with gr.Column():
                # Output section
                gr.Markdown("### Output")
                output_text = gr.Textbox(
                    label="Translation",
                    lines=3,
                    interactive=False  # Make it read-only
                )
                target_lang = gr.Dropdown(
                    choices=language_options,
                    label="Target Language",
                    value="Darija"  # default value
                )
        
         # Translate button
         translate_btn = gr.Button("Translate", variant="primary")
        
         # Clear button
         clear_btn = gr.Button("Clear", variant="secondary")
        
         # Set up the translation action
         translate_btn.click(
            fn=chat,
            inputs=[input_text, source_lang, target_lang],
            outputs=output_text
         )
        
         # Set up clear action
         clear_btn.click(
            fn=lambda: ("", ""),
            inputs=[],
            outputs=[input_text, output_text]
         )
        
         # Optional: Add examples
         gr.Examples(
            examples=[
                ["Bghit nakul", "Darija", "English"],
                ["I want to eat", "English", "Darija"],
                ["أريد أن آكل", "Arabic", "English"],
            ],
            inputs=[input_text, source_lang, target_lang],
         )
    
    return interface

In [15]:
interface = create_translator_interface()
interface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
